# 🧪 Exemplo de Aplicação com Conexão a Banco de Dados

Este notebook demonstra como criar uma aplicação simples em Python que interage com um banco de dados **PostgreSQL** utilizando bibliotecas como **Pandas**, **SQLAlchemy**, **Panel**, entre outras. A interface gráfica permite consultar, inserir, atualizar e excluir registros da tabela `pessoa`.

---

## 🛠️ Organização do Projeto e Ambiente Virtual

Para garantir isolamento e facilitar a manutenção do ambiente Python, é **fortemente recomendado** utilizar um **ambiente virtual**. Isso evita conflitos entre dependências de diferentes projetos.

### ✅ Criar ambiente virtual (Linux, macOS ou WSL)

```bash
python3 -m venv venv
source venv/bin/activate
```

### ✅ Criar ambiente virtual (Windows)

```bash
python -m venv venv
venv\Scripts\activate
```

---

## 📦 `requirements.txt` — Instalação de Dependências

Crie um arquivo chamado `requirements.txt` no diretório do projeto com o seguinte conteúdo:

```txt
pandas
sqlalchemy
psycopg2-binary
panel
python-dotenv
```

### ✅ Instalar as dependências com o pip

```bash
pip install -r requirements.txt
```

---

## 🔐 Utilizando o `.env` para Conexão com o Banco de Dados

Para proteger informações sensíveis como usuário, senha e nome do banco, recomendamos armazenar esses dados em um **arquivo `.env`**, que não deve ser incluído no repositório de código (como o GitHub).

### ✅ Exemplo de conteúdo do arquivo `.env`

```dotenv
DB_HOST=localhost
DB_NAME=fbd-conexao
DB_USER=postgres
DB_PASS=root
```

---

## 📎 `.env.example`: Informando a Estrutura Esperada

Crie também um arquivo chamado **`.env.example`**, que serve como modelo para outras pessoas saberem quais variáveis são esperadas no projeto (sem conter dados reais).

Esse arquivo **pode ser incluído no repositório**, pois não contém credenciais, apenas a estrutura necessária.

---

## 🚫 Protegendo Dados com `.gitignore`

Adicione os seguintes itens no seu arquivo `.gitignore` para evitar subir arquivos sensíveis ao repositório:

---

## 🧑‍💻 Rodando a Aplicação

Após configurar o banco de dados, instalar as dependências e ativar o ambiente virtual, você poderá executar a aplicação com:

```bash
panel serve nome_do_arquivo.py --autoreload --show
```

Ou, se estiver usando Jupyter Notebook, poderá importar as funções diretamente e utilizar a interface com `pn.panel(...)`.

---

In [ ]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
import pandas as pd
import psycopg2 as pg
import sqlalchemy
import panel as pn
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from sqlalchemy import create_engine


In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx_string = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'
engine = sqlalchemy.create_engine(cnx_string)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from usuario;" 
df = pd.read_sql_query(query, engine)

df.head()

In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    # Create fresh button instances every time
    btn_usuarios = pn.widgets.Button(name="Usuários", button_type="primary", width=250)    
    btn_campanhas = pn.widgets.Button(name="Campanhas", button_type="primary", width=250)
    btn_ordens = pn.widgets.Button(name="Ordens", button_type="primary", width=250)
    btn_doacoes = pn.widgets.Button(name="Doações", button_type="primary", width=250)
    btn_itens = pn.widgets.Button(name="Itens", button_type="primary", width=250)
    btn_graficos = pn.widgets.Button(name="Gráficos", button_type="danger", width=250)
    
    # Connect their clicks
    btn_usuarios.on_click(lambda event: go_to_usuarios())    
    btn_campanhas.on_click(lambda event: go_to_campanhas())
    btn_ordens.on_click(lambda event: go_to_ordens())
    btn_doacoes.on_click(lambda event: go_to_doacoes())
    btn_itens.on_click(lambda event: go_to_itens())
    btn_graficos.on_click(lambda event: go_to_graficos())
        
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),        
        pn.pane.Markdown("Escolha uma opção:", styles={"font-size": "14pt"}),        
        btn_usuarios,        
        btn_campanhas,
        btn_ordens,
        btn_doacoes,
        btn_itens,                
        btn_graficos,
        align="center",
        sizing_mode="stretch_width",
        margin=(50,50,50,50)
    )
    return home

In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Tela Inicial', button_type='default')
    
    def voltar_action(event):
        main_area.clear()
        main_area.append(create_home_view())
    
    btn_voltar_home.on_click(voltar_action)
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag = ''

# Widget de seleção de tipo de pessoa
tipo_pessoa_widget = pn.widgets.RadioBoxGroup(
    name="Tipo de Pessoa",
    options=["Doador", "Beneficiário", "Instituição"],
    inline=False
)

# Widgets de entrada de dados
def create_form_widgets():
    return {
        "tipo_pessoa": tipo_pessoa_widget,
        "nome": pn.widgets.TextInput(
            name="Nome",
            placeholder="Digite o nome",
            sizing_mode="stretch_width"
        ),
        "cpf_cnpj": pn.widgets.TextInput(
            name="CPF/CNPJ",
            placeholder="Digite o CPF ou CNPJ",
            sizing_mode="stretch_width"
        ),
        "email": pn.widgets.TextInput(
            name="Email",
            placeholder="Digite o email",
            sizing_mode="stretch_width"
        ),
        "celular": pn.widgets.TextInput(
            name="Celular",
            placeholder="(XX) XXXXX-XXXX",
            sizing_mode="stretch_width"
        ),
        "senha": pn.widgets.PasswordInput(
            name="Senha",
            placeholder="Digite uma senha segura",
            sizing_mode="stretch_width"
        ),
        "datanasc": pn.widgets.DatePicker(
            name="Data de Nascimento",
            visible=False
        ),
        "rua": pn.widgets.TextInput(
            name="Rua",
            placeholder="Digite a rua",
            sizing_mode="stretch_width"
        ),
        "numero": pn.widgets.TextInput(
            name="Número",
            placeholder="Digite o número",
            sizing_mode="stretch_width"
        ),
        "bairro": pn.widgets.TextInput(
            name="Bairro",
            placeholder="Digite o bairro",
            sizing_mode="stretch_width"
        ),
        "cidade": pn.widgets.TextInput(
            name="Cidade",
            placeholder="Digite a cidade",
            sizing_mode="stretch_width"
        ),
        "estado": pn.widgets.TextInput(
            name="Estado",
            placeholder="Digite o estado",
            sizing_mode="stretch_width"
        ),
        "cep": pn.widgets.TextInput(
            name="CEP",
            placeholder="XXXXXXXX",
            sizing_mode="stretch_width"
        )
    }



In [ ]:
# Botões de ações CRUD
btn_consultar = pn.widgets.Button(name='Consultar', button_type='primary')
btn_inserir = pn.widgets.Button(name='Inserir', button_type='success')
btn_excluir = pn.widgets.Button(name='Excluir', button_type='danger')
btn_atualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

btn_consultar.margin = (0, 1, 0, 10)
btn_inserir.margin = (0, 1, 0, 0)
btn_atualizar.margin = (0, 1, 0, 0)
btn_excluir.margin = (0, 1, 0, 0)

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Tela Inicial', button_type='default')


In [ ]:
# Funções CRUD adaptadas para a nova estrutura

def queryAll():
    """
    Consulta todos os usuários e retorna um Tabulator.
    """
    query = """
    SELECT u.cpf_cnpj, u.nome, u.email, u.celular, 
           CASE 
               WHEN d.cpf_cnpj_d IS NOT NULL THEN 'Doador'
               WHEN b.cpf_cnpj_b IS NOT NULL THEN 'Beneficiário'
               WHEN i.cpf_cnpj_i IS NOT NULL THEN 'Instituição'
           END AS tipo
    FROM usuario u
    LEFT JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
    LEFT JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
    LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
    ORDER BY u.nome
    """
    df = pd.read_sql_query(query, engine)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def on_consultar(w):
    """Consulta pelo CPF/CNPJ ou retorna todos."""
    try:
        cpf_cnpj = w["cpf_cnpj"].value
        if cpf_cnpj.strip() == '':
            return queryAll()
        
        query = """
        SELECT u.cpf_cnpj, u.nome, u.email, u.celular, 
               CASE 
                   WHEN d.cpf_cnpj_d IS NOT NULL THEN 'Doador'
                   WHEN b.cpf_cnpj_b IS NOT NULL THEN 'Beneficiário'
                   WHEN i.cpf_cnpj_i IS NOT NULL THEN 'Instituição'
               END AS tipo
        FROM usuario u
        LEFT JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
        LEFT JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
        LEFT JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
        WHERE u.cpf_cnpj = %s
        """
        df = pd.read_sql_query(query, engine, params=(cpf_cnpj,))
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir(w):
    """Insere um novo usuário com seu tipo específico."""
    try:
        cursor = con.cursor()
        tipo = w["tipo_pessoa"].value
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj or not w["nome"].value or not w["senha"].value:
            return pn.pane.Alert('CPF/CNPJ, Nome e Senha são obrigatórios!', alert_type='warning')
        
        # Insere na tabela usuário com senha
        cursor.execute(
            """INSERT INTO usuario(cpf_cnpj, nome, email, celular, rua, numero, bairro, 
                                   cidade, estado, cep, senha) 
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
            (cpf_cnpj, w["nome"].value, w["email"].value, w["celular"].value,
             w["rua"].value, w["numero"].value, w["bairro"].value, 
             w["cidade"].value, w["estado"].value, w["cep"].value, w["senha"].value)
        )
        
        # Insere na tabela específica
        if tipo == "Doador":
            cursor.execute(
                "INSERT INTO doador(cpf_cnpj_d, data_nascimento) VALUES (%s, %s)",
                (cpf_cnpj, w["datanasc"].value)
            )
        elif tipo == "Beneficiário":
            cursor.execute(
                "INSERT INTO beneficiario(cpf_cnpj_b, data_nascimento) VALUES (%s, %s)",
                (cpf_cnpj, w["datanasc"].value)
            )
        elif tipo == "Instituição":
            cursor.execute(
                "INSERT INTO instituicao(cpf_cnpj_i) VALUES (%s)",
                (cpf_cnpj,)
            )
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário inserido com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao inserir: {str(e)}', alert_type='danger')

def on_atualizar(w):
    """Atualiza dados do usuário."""
    try:
        cursor = con.cursor()
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj:
            return pn.pane.Alert('CPF/CNPJ é obrigatório para atualizar!', alert_type='warning')
        
        # Se senha foi preenchida, atualiza também a senha
        if w["senha"].value:
            cursor.execute(
                """UPDATE usuario SET nome=%s, email=%s, celular=%s, rua=%s, numero=%s, 
                                   bairro=%s, cidade=%s, estado=%s, cep=%s, senha=%s
                   WHERE cpf_cnpj=%s""",
                (w["nome"].value, w["email"].value, w["celular"].value,
                 w["rua"].value, w["numero"].value, w["bairro"].value,
                 w["cidade"].value, w["estado"].value, w["cep"].value, w["senha"].value, cpf_cnpj)
            )
        else:
            cursor.execute(
                """UPDATE usuario SET nome=%s, email=%s, celular=%s, rua=%s, numero=%s, 
                                   bairro=%s, cidade=%s, estado=%s, cep=%s
                   WHERE cpf_cnpj=%s""",
                (w["nome"].value, w["email"].value, w["celular"].value,
                 w["rua"].value, w["numero"].value, w["bairro"].value,
                 w["cidade"].value, w["estado"].value, w["cep"].value, cpf_cnpj)
            )
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário atualizado com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao atualizar: {str(e)}', alert_type='danger')

def on_excluir(w):
    """Exclui um usuário e suas referências."""
    try:
        cursor = con.cursor()
        cpf_cnpj = w["cpf_cnpj"].value
        
        if not cpf_cnpj:
            return pn.pane.Alert('CPF/CNPJ é obrigatório para excluir!', alert_type='warning')
        
        # Deleta das tabelas específicas
        cursor.execute("DELETE FROM doador WHERE cpf_cnpj_d=%s", (cpf_cnpj,))
        cursor.execute("DELETE FROM beneficiario WHERE cpf_cnpj_b=%s", (cpf_cnpj,))
        cursor.execute("DELETE FROM instituicao WHERE cpf_cnpj_i=%s", (cpf_cnpj,))
        
        # Deleta da tabela usuário
        cursor.execute("DELETE FROM usuario WHERE cpf_cnpj=%s", (cpf_cnpj,))
        
        con.commit()
        cursor.close()
        pn.state.notifications.success('Usuário excluído com sucesso!')
        return queryAll()
    except Exception as e:
        con.rollback()
        cursor.close()
        return pn.pane.Alert(f'Erro ao excluir: {str(e)}', alert_type='danger')


In [ ]:
def table_creator(cons, ins, atu, exc, w):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    if cons: 
        return on_consultar(w)
    if ins: 
        resultado = on_inserir(w)
        # Reseta formulário após sucesso
        if isinstance(resultado, pn.widgets.Tabulator):
            for key in w:
                if hasattr(w[key], 'value'):
                    w[key].value = "" if key != "datanasc" else None
        return resultado
    if atu: 
        return on_atualizar(w)
    if exc: 
        return on_excluir(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

# Conecta os botões à tabela interativa
form_widgets = create_form_widgets()

def create_interactive_table():
    return pn.bind(
        table_creator,
        btn_consultar,
        btn_inserir,
        btn_atualizar,
        btn_excluir,
        form_widgets
    )


In [ ]:
def create_user_view(w):
    """
    Cria a tela do CRUD com layout bonito.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown("### 👥 Gerenciador de Usuários"),
            create_btn_voltar(),
            w["tipo_pessoa"],
            pn.Spacer(height=10),
            w["nome"],
            w["cpf_cnpj"],
            w["email"],
            w["celular"],
            w["senha"],
            w["datanasc"],
            pn.Spacer(height=10),
            pn.pane.Markdown("**Endereço**"),
            w["rua"],
            w["numero"],
            w["bairro"],
            w["cidade"],
            w["estado"],
            w["cep"],
            pn.Spacer(height=15),
            pn.Row(btn_consultar, btn_inserir, btn_atualizar, btn_excluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table(), sizing_mode='stretch_both')
    )
    return crud_layout

def go_to_usuarios():
    main_area.clear()
    main_area.append(create_user_view(form_widgets))

In [ ]:
from graficos import create_graficos_view as _create_graficos_view

def create_graficos_view():
    """Wrapper que usa o `engine` do notebook e o `create_btn_voltar`."""
    try:
        return _create_graficos_view(engine, create_btn_voltar)
    except Exception as e:
        return pn.pane.Alert(f'Erro ao montar view de gráficos: {str(e)}', alert_type='danger')

def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# ===== TELA DE CAMPANHAS =====

def get_instituicoes():
    """Busca todas as instituições cadastradas"""
    try:
        query = """
        SELECT DISTINCT u.cpf_cnpj, u.nome
        FROM usuario u
        JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
        ORDER BY u.nome
        """
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['nome'], df['cpf_cnpj']))
    except Exception as e:
        return {}

# Widgets para campanhas
instituicoes_dict = get_instituicoes()
instituicoes_select = pn.widgets.TextInput(
    name="Instituição",
    placeholder="Digite o nome da instituição",
    sizing_mode="stretch_width",
    value=""
)

nome_campanha = pn.widgets.TextInput(
    name="Nome da Campanha",
    placeholder="Digite o nome da campanha",
    sizing_mode="stretch_width"
)

data_inicio = pn.widgets.DatePicker(
    name="Data de Início",
    sizing_mode="stretch_width"
)

data_fim = pn.widgets.DatePicker(
    name="Data de Término",
    sizing_mode="stretch_width"
)

status_campanha = pn.widgets.Select(
    name="Status",
    options=["N/A", "Planejada", "Ativa", "Inativa", "Concluída"],
    value="N/A",
    sizing_mode="stretch_width"
)

# Botões de ações para campanhas
btn_consultar_campanhas = pn.widgets.Button(name='Consultar', button_type='primary')
btn_inserir_campanhas = pn.widgets.Button(name='Inserir', button_type='success')
btn_atualizar_campanhas = pn.widgets.Button(name='Atualizar', button_type='warning')
btn_excluir_campanhas = pn.widgets.Button(name='Excluir', button_type='danger')
btn_ativar_inativar = pn.widgets.Button(name='Ativar/Inativar Campanha', button_type='danger')

btn_consultar_campanhas.margin = (0, 1, 0, 10)
btn_inserir_campanhas.margin = (0, 1, 0, 0)
btn_atualizar_campanhas.margin = (0, 1, 0, 0)
btn_excluir_campanhas.margin = (0, 1, 0, 0)

def on_consultar_campanhas(event):
    """Consulta campanhas com filtro por instituição, nome, datas e status. Se vazio, retorna todas."""
    try:
        termo_instituicao = instituicoes_select.value.strip()
        termo_campanha = nome_campanha.value.strip()
        data_ini = data_inicio.value
        data_fim_val = data_fim.value
        status_selecionado = status_campanha.value
        
        # Construir a query base
        query_base = """
        SELECT c.id_campanha, u.nome AS instituicao, c.nome, 
               c.data_inicio, c.data_fim, c.status
        FROM campanha c
        JOIN instituicao i ON c.cpf_cnpj_i = i.cpf_cnpj_i
        JOIN usuario u ON i.cpf_cnpj_i = u.cpf_cnpj
        WHERE 1=1
        """
        
        params = []
        
        # Filtros de Texto
        if termo_instituicao:
            query_base += " AND u.nome ILIKE %s"
            params.append(f"%{termo_instituicao}%")
        
        if termo_campanha:
            query_base += " AND c.nome ILIKE %s"
            params.append(f"%{termo_campanha}%")
        
        # Filtro de Status (só aplica se não for "N/A")
        if status_selecionado != "N/A":
            query_base += " AND c.status = %s"
            params.append(status_selecionado)
        
        # --- Lógica para filtros de data ---
        if data_ini and data_fim_val:
            # Intervalo: data_inicio >= inicio E data_fim <= fim
            query_base += " AND c.data_inicio >= %s::date AND c.data_fim <= %s::date"
            params.append(str(data_ini))
            params.append(str(data_fim_val))

        elif data_ini:
            # Data exata no início
            query_base += " AND c.data_inicio = %s::date"
            params.append(str(data_ini))

        elif data_fim_val:
            # Data exata no fim
            query_base += " AND c.data_fim = %s::date"
            params.append(str(data_fim_val))
        
        query_base += " ORDER BY c.data_inicio DESC"
        
        # Executa a query
        if params:
            df = pd.read_sql_query(query_base, engine, params=tuple(params))
        else:
            df = pd.read_sql_query(query_base, engine)
        
        if df.empty:
            return pn.pane.Alert('❌ Nenhuma campanha encontrada com esses critérios de busca', alert_type='warning')
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'❌ Erro ao consultar: {str(e)}', alert_type='danger')

def on_criar_campanha(event):
    """Cria uma nova campanha com status Planejada"""
    try:
        termo_busca = instituicoes_select.value.strip()
        
        if not termo_busca or not nome_campanha.value:
            pn.state.notifications.error('❌ Instituição e Nome são obrigatórios!')
            return
        
        if not data_inicio.value or not data_fim.value:
            pn.state.notifications.error('❌ Data de Início e Data de Término são obrigatórias!')
            return
        
        # Busca a instituição com ILIKE (busca parcial, case-insensitive)
        cursor = con.cursor()
        cursor.execute(
            "SELECT u.cpf_cnpj, u.nome FROM usuario u JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i WHERE u.nome ILIKE %s",
            (f"%{termo_busca}%",)
        )
        results = cursor.fetchall()
        
        if not results:
            cursor.close()
            pn.state.notifications.error('❌ Instituição não encontrada no banco! Primeiro cadastre a instituição na tela de CRUD.')
            return
        
        if len(results) > 1:
            cursor.close()
            nomes = ', '.join([r[1] for r in results])
            pn.state.notifications.error(f'❌ Múltiplas instituições encontradas: {nomes}. Digite o nome completo ou seja mais específico.')
            return
        
        cpf_cnpj_instituicao = results[0][0]
        cursor.close()
        
        # INSERT INTO campanha
        cursor = con.cursor()
        cursor.execute(
            """INSERT INTO campanha(cpf_cnpj_i, nome, data_inicio, data_fim, status) 
               VALUES (%s, %s, %s, %s, 'Planejada')""",
            (cpf_cnpj_instituicao, nome_campanha.value,
             data_inicio.value, data_fim.value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('✅ Campanha criada com sucesso com status Planejada!')
        
        # Limpar campos
        instituicoes_select.value = ""
        nome_campanha.value = ""
        data_inicio.value = None
        data_fim.value = None
        status_campanha.value = "N/A"
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'❌ Erro ao criar campanha: {str(e)}')

def on_atualizar_campanha(event):
    """Atualiza uma campanha existente com novos dados"""
    try:
        termo_busca = instituicoes_select.value.strip()
        
        if not termo_busca or not nome_campanha.value:
            pn.state.notifications.warning('Instituição e Nome da Campanha são obrigatórios!')
            return
        
        cursor = con.cursor()
        
        # Busca o CPF/CNPJ da instituição
        cursor.execute(
            "SELECT u.cpf_cnpj FROM usuario u JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i WHERE u.nome = %s",
            (termo_busca,)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Instituição não encontrada!')
            return
        
        cpf_cnpj = result[0]
        
        # Verifica se a campanha existe
        cursor.execute(
            "SELECT id_campanha FROM campanha WHERE cpf_cnpj_i=%s AND nome=%s",
            (cpf_cnpj, nome_campanha.value)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Campanha não encontrada!')
            return
        
        # Atualiza a campanha
        cursor.execute(
            """UPDATE campanha SET data_inicio=%s, data_fim=%s, status=%s
               WHERE cpf_cnpj_i=%s AND nome=%s""",
            (data_inicio.value, data_fim.value, status_campanha.value,
             cpf_cnpj, nome_campanha.value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Campanha atualizada com sucesso!')
        
        # Limpar campos
        instituicoes_select.value = ""
        data_inicio.value = None
        data_fim.value = None
        status_campanha.value = "N/A"
        nome_campanha.value = ""
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao atualizar: {str(e)}')

def on_excluir_campanha(event):
    """Exclui uma campanha"""
    try:
        termo_busca = instituicoes_select.value.strip()
        
        if not termo_busca or not nome_campanha.value:
            pn.state.notifications.warning('Instituição e Nome da Campanha são obrigatórios!')
            return
        
        cursor = con.cursor()
        
        # Busca o CPF/CNPJ da instituição
        cursor.execute(
            "SELECT u.cpf_cnpj FROM usuario u JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i WHERE u.nome = %s",
            (termo_busca,)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Instituição não encontrada!')
            return
        
        cpf_cnpj = result[0]
        
        # Verifica se a campanha existe
        cursor.execute(
            "SELECT id_campanha FROM campanha WHERE cpf_cnpj_i=%s AND nome=%s",
            (cpf_cnpj, nome_campanha.value)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Campanha não encontrada!')
            return
        
        # Deleta a campanha
        cursor.execute(
            "DELETE FROM campanha WHERE cpf_cnpj_i=%s AND nome=%s",
            (cpf_cnpj, nome_campanha.value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Campanha excluída com sucesso!')
        
        # Limpar campos
        instituicoes_select.value = ""
        nome_campanha.value = ""
        data_inicio.value = None
        data_fim.value = None
        status_campanha.value = "N/A"
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao excluir: {str(e)}')

def on_ativar_inativar(event):
    """Alterna o status da campanha entre Ativa e Inativa"""
    try:
        termo_busca = instituicoes_select.value.strip()
        
        if not termo_busca or not nome_campanha.value:
            pn.state.notifications.warning('Instituição e Nome da Campanha são obrigatórios!')
            return
        
        cursor = con.cursor()
        
        # Busca o CPF/CNPJ da instituição
        cursor.execute(
            "SELECT u.cpf_cnpj FROM usuario u JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i WHERE u.nome = %s",
            (termo_busca,)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Instituição não encontrada!')
            return
        
        cpf_cnpj = result[0]
        
        cursor.execute(
            "SELECT status FROM campanha WHERE cpf_cnpj_i=%s AND nome=%s",
            (cpf_cnpj, nome_campanha.value)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Campanha não encontrada!')
            return
        
        status_atual = result[0]
        novo_status = 'Inativa' if status_atual == 'Ativa' else 'Ativa'
        
        cursor.execute(
            "UPDATE campanha SET status=%s WHERE cpf_cnpj_i=%s AND nome=%s",
            (novo_status, cpf_cnpj, nome_campanha.value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success(f'Campanha alterada para {novo_status}!')
        
        # Atualizar o widget e limpar
        status_campanha.value = novo_status
        instituicoes_select.value = ""
        nome_campanha.value = ""
        data_inicio.value = None
        data_fim.value = None
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao alterar status: {str(e)}')

btn_consultar_campanhas.on_click(on_consultar_campanhas)
btn_inserir_campanhas.on_click(on_criar_campanha)
btn_atualizar_campanhas.on_click(on_atualizar_campanha)
btn_excluir_campanhas.on_click(on_excluir_campanha)
btn_ativar_inativar.on_click(on_ativar_inativar)

def create_campanhas_view():
    """Cria a tela de campanhas"""
    campanhas_layout = pn.Column(
        pn.pane.Markdown("### 📢 Gerenciador de Campanhas"),
        create_btn_voltar(),
        pn.pane.Markdown("**Preencha os dados para gerenciar campanhas**"),
        instituicoes_select,
        nome_campanha,
        data_inicio,
        data_fim,
        status_campanha,
        pn.Row(btn_consultar_campanhas, btn_inserir_campanhas, btn_atualizar_campanhas, btn_excluir_campanhas),
        pn.Row(btn_ativar_inativar),
        pn.Spacer(height=1),
        pn.bind(on_consultar_campanhas, btn_consultar_campanhas),
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20, 20, 20, 20),
    )
    return campanhas_layout

def go_to_campanhas():
    main_area.clear()
    main_area.append(create_campanhas_view())

In [ ]:
# App principal, com título e área dinâmica
# Inicializa com a home
main_area.append(create_home_view())

pn.Column(
    pn.pane.Markdown("# Sistema de Doações", styles={"margin": "50px 0px 0px 50px"}),
    main_area
).servable()


In [ ]:
def get_doacoes_list():
    try:
        df = pd.read_sql_query("SELECT id_doacao FROM doacao ORDER BY id_doacao DESC", engine)
        return df['id_doacao'].tolist()
    except:
        return []

def get_beneficiarios_list():
    """Busca todos os beneficiários cadastrados"""
    try:
        query = """
        SELECT DISTINCT u.cpf_cnpj, u.nome
        FROM usuario u
        JOIN beneficiario b ON u.cpf_cnpj = b.cpf_cnpj_b
        ORDER BY u.nome
        """
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['nome'], df['cpf_cnpj']))
    except:
        return {}

def get_instituicoes_list():
    """Busca todas as instituições cadastradas"""
    try:
        query = """
        SELECT DISTINCT u.cpf_cnpj, u.nome
        FROM usuario u
        JOIN instituicao i ON u.cpf_cnpj = i.cpf_cnpj_i
        ORDER BY u.nome
        """
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['nome'], df['cpf_cnpj']))
    except:
        return {}

def create_ordem_widgets():
    beneficiarios = get_beneficiarios_list()
    instituicoes = get_instituicoes_list()
    return {
        "id_ordem": pn.widgets.IntInput(name="ID da Ordem", value=0, start=0),
        "cpf_cnpj_b": pn.widgets.Select(name="Beneficiário", options=beneficiarios, sizing_mode="stretch_width"),
        "cpf_cnpj_i": pn.widgets.Select(name="Instituição", options=instituicoes, sizing_mode="stretch_width"),
        "status_ordem": pn.widgets.Select(name="Status", options=["Pendente", "Em Processamento", "Concluída", "Cancelada"]),
        "data_hora_criacao": pn.widgets.DatetimePicker(name="Data/Hora de Criação"),
        "data_hora_retirada": pn.widgets.DatetimePicker(name="Data/Hora de Retirada"),
        "rua": pn.widgets.TextInput(name="Rua", placeholder="Digite a rua", sizing_mode="stretch_width"),
        "numero": pn.widgets.TextInput(name="Número", placeholder="Digite o número", sizing_mode="stretch_width"),
        "bairro": pn.widgets.TextInput(name="Bairro", placeholder="Digite o bairro", sizing_mode="stretch_width"),
        "cidade": pn.widgets.TextInput(name="Cidade", placeholder="Digite a cidade", sizing_mode="stretch_width"),
        "estado": pn.widgets.TextInput(name="Estado", placeholder="Digite o estado", sizing_mode="stretch_width"),
        "cep": pn.widgets.TextInput(name="CEP", placeholder="XXXXXXXX", sizing_mode="stretch_width"),
    }

ordem_widgets = create_ordem_widgets()
btn_consultar_ordens = pn.widgets.Button(name='Consultar', button_type='primary')
btn_inserir_ordens = pn.widgets.Button(name='Inserir', button_type='success')
btn_atualizar_ordens = pn.widgets.Button(name='Atualizar', button_type='warning')
btn_deletar_ordens = pn.widgets.Button(name='Deletar', button_type='danger')

btn_consultar_ordens.margin = (0, 1, 0, 10)
btn_inserir_ordens.margin = (0, 1, 0, 0)
btn_atualizar_ordens.margin = (0, 1, 0, 0)
btn_deletar_ordens.margin = (0, 1, 0, 0)

def query_all_ordens():
    try:
        query = """
        SELECT o.id_ordem_doacao, u_b.nome AS beneficiario, u_i.nome AS instituicao, o.status, 
               o.data_hora_criacao, o.data_hora_retirada, o.rua, o.numero, o.bairro, o.cidade, o.estado, o.cep,
               STRING_AGG(DISTINCT i.id_doacao::text, ', ') AS doacoes_associadas
        FROM ordem_de_doacao o
        LEFT JOIN beneficiario b ON o.cpf_cnpj_b = b.cpf_cnpj_b
        LEFT JOIN usuario u_b ON b.cpf_cnpj_b = u_b.cpf_cnpj
        LEFT JOIN instituicao inst ON o.cpf_cnpj_i = inst.cpf_cnpj_i
        LEFT JOIN usuario u_i ON inst.cpf_cnpj_i = u_i.cpf_cnpj
        LEFT JOIN item_doacao i ON o.id_ordem_doacao = i.id_ordem_doacao
        GROUP BY o.id_ordem_doacao, u_b.nome, u_i.nome, o.status, 
                 o.data_hora_criacao, o.data_hora_retirada, o.rua, o.numero, 
                 o.bairro, o.cidade, o.estado, o.cep
        ORDER BY o.data_hora_criacao DESC
        """
        df = pd.read_sql_query(query, engine)
        if df.empty:
            return pn.pane.Alert('Nenhuma ordem encontrada', alert_type='info')
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'❌ Erro ao consultar ordens: {str(e)}', alert_type='danger')

def on_consultar_ordens(w):
    try:
        query = """
        SELECT o.id_ordem_doacao, u_b.nome AS beneficiario, u_i.nome AS instituicao, o.status, 
               o.data_hora_criacao, o.data_hora_retirada, o.rua, o.numero, o.bairro, o.cidade, o.estado, o.cep,
               STRING_AGG(DISTINCT i.id_doacao::text, ', ') AS doacoes_associadas
        FROM ordem_de_doacao o
        LEFT JOIN beneficiario b ON o.cpf_cnpj_b = b.cpf_cnpj_b
        LEFT JOIN usuario u_b ON b.cpf_cnpj_b = u_b.cpf_cnpj
        LEFT JOIN instituicao inst ON o.cpf_cnpj_i = inst.cpf_cnpj_i
        LEFT JOIN usuario u_i ON inst.cpf_cnpj_i = u_i.cpf_cnpj
        LEFT JOIN item_doacao i ON o.id_ordem_doacao = i.id_ordem_doacao
        GROUP BY o.id_ordem_doacao, u_b.nome, u_i.nome, o.status, 
                 o.data_hora_criacao, o.data_hora_retirada, o.rua, o.numero, 
                 o.bairro, o.cidade, o.estado, o.cep
        ORDER BY o.data_hora_criacao DESC
        """
        df = pd.read_sql_query(query, engine)
        if df.empty:
            return pn.pane.Alert('Nenhuma ordem encontrada', alert_type='info')
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'❌ Erro ao consultar ordens: {str(e)}', alert_type='danger')

def on_inserir_ordens(w):
    try:
        if not w["cpf_cnpj_b"].value or not w["cpf_cnpj_i"].value or not w["status_ordem"].value:
            pn.state.notifications.warning('Beneficiário, Instituição e Status são obrigatórios!')
            return query_all_ordens()
        
        cursor = con.cursor()
        cursor.execute(
            """INSERT INTO ordem_de_doacao(cpf_cnpj_b, cpf_cnpj_i, status, data_hora_criacao, data_hora_retirada, rua, numero, bairro, cidade, estado, cep) 
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
            (w["cpf_cnpj_b"].value, w["cpf_cnpj_i"].value, w["status_ordem"].value,
             w["data_hora_criacao"].value, w["data_hora_retirada"].value,
             w["rua"].value, w["numero"].value, w["bairro"].value, w["cidade"].value, w["estado"].value, w["cep"].value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Ordem inserida com sucesso! Use a tela de Itens para associar doações.')
        return query_all_ordens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'❌ Erro ao inserir ordem: {str(e)}', alert_type='danger')

def on_atualizar_ordens(w):
    if not w["id_ordem"].value:
        pn.state.notifications.warning('ID da Ordem é obrigatório!')
        return query_all_ordens()
    
    try:
        cursor = con.cursor()
        cursor.execute(
            """UPDATE ordem_de_doacao SET cpf_cnpj_b=%s, cpf_cnpj_i=%s, status=%s, 
                                       data_hora_criacao=%s, data_hora_retirada=%s, rua=%s, numero=%s, 
                                       bairro=%s, cidade=%s, estado=%s, cep=%s 
               WHERE id_ordem_doacao=%s""",
            (w["cpf_cnpj_b"].value, w["cpf_cnpj_i"].value, w["status_ordem"].value,
             w["data_hora_criacao"].value, w["data_hora_retirada"].value,
             w["rua"].value, w["numero"].value, w["bairro"].value, w["cidade"].value, w["estado"].value, w["cep"].value,
             w["id_ordem"].value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Ordem atualizada com sucesso!')
        return query_all_ordens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'❌ Erro ao atualizar ordem: {str(e)}', alert_type='danger')

def on_deletar_ordens(w):
    if not w["id_ordem"].value:
        pn.state.notifications.warning('ID da Ordem é obrigatório!')
        return query_all_ordens()
    
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM ordem_de_doacao WHERE id_ordem_doacao=%s", (w["id_ordem"].value,))
        con.commit()
        cursor.close()
        pn.state.notifications.success('Ordem deletada com sucesso!')
        return query_all_ordens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'❌ Erro ao deletar ordem: {str(e)}', alert_type='danger')

def table_creator_ordens(cons, ins, atu, dele, w):
    if cons: return on_consultar_ordens(w)
    if ins: return on_inserir_ordens(w)
    if atu: return on_atualizar_ordens(w)
    if dele: return on_deletar_ordens(w)
    return query_all_ordens()

def create_interactive_table_ordens():
    return pn.Column(
        query_all_ordens(),
        pn.bind(table_creator_ordens, btn_consultar_ordens, btn_inserir_ordens, btn_atualizar_ordens, btn_deletar_ordens, ordem_widgets),
        sizing_mode='stretch_both'
    )

def create_ordens_view(w):
    table_area = pn.Column(query_all_ordens(), sizing_mode='stretch_both')
    
    def update_table(c, i, a, d):
        if c or i or a or d:
            table_area.clear()
            table_area.append(table_creator_ordens(c, i, a, d, w))
    
    pn.bind(update_table, btn_consultar_ordens, btn_inserir_ordens, btn_atualizar_ordens, btn_deletar_ordens, watch=True)
    
    return pn.Row(
        pn.Column(
            pn.pane.Markdown("### 📦 Gerenciar Ordens de Doação"),
            create_btn_voltar(),
            pn.pane.Markdown("**Dados da Ordem**"),
            w["id_ordem"],
            w["cpf_cnpj_b"],
            w["cpf_cnpj_i"],
            w["status_ordem"],
            w["data_hora_criacao"],
            w["data_hora_retirada"],
            pn.pane.Markdown("**Endereço de Entrega**"),
            w["rua"],
            w["numero"],
            w["bairro"],
            w["cidade"],
            w["estado"],
            w["cep"],
            pn.Row(btn_consultar_ordens, btn_inserir_ordens, btn_atualizar_ordens, btn_deletar_ordens),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        table_area
    )

def go_to_ordens():
    main_area.clear()
    main_area.append(create_ordens_view(ordem_widgets))

# ===== TELA DE ITENS DE DOAÇÃO =====

def get_ordens_list():
    """Busca todas as ordens de doação disponíveis"""
    try:
        query = """
        SELECT DISTINCT o.id_ordem_doacao, 
               CONCAT(o.id_ordem_doacao, ' - ', u.nome, ' (', o.status, ')')
        FROM ordem_de_doacao o
        LEFT JOIN beneficiario b ON o.cpf_cnpj_b = b.cpf_cnpj_b
        LEFT JOIN usuario u ON b.cpf_cnpj_b = u.cpf_cnpj
        ORDER BY o.id_ordem_doacao DESC
        """
        df = pd.read_sql_query(query, engine)
        if df.empty:
            return {}
        return dict(zip(df.iloc[:, 1], df.iloc[:, 0]))
    except:
        return {}

def create_itens_widgets():
    doacoes = get_doacoes_list()
    ordens = get_ordens_list()
    return {
        "id_item": pn.widgets.IntInput(name="ID do Item", value=0, start=0),
        "id_doacao": pn.widgets.Select(name="Doação (obrigatório)", options=doacoes, sizing_mode="stretch_width"),
        "id_ordem_doacao": pn.widgets.Select(name="Ordem de Doação (opcional)", options=ordens, sizing_mode="stretch_width"),
        "nome": pn.widgets.TextInput(name="Nome", placeholder="Nome do item", sizing_mode="stretch_width"),
        "descricao": pn.widgets.TextInput(name="Descrição", placeholder="Descrição", sizing_mode="stretch_width"),
        "estado_conservacao": pn.widgets.Select(name="Estado", options=["Novo", "Bom", "Regular", "Ruim"]),
        "peso": pn.widgets.FloatInput(name="Peso (kg)", value=0.0),
        "volume": pn.widgets.FloatInput(name="Volume (m³)", value=0.0),
        "tamanho": pn.widgets.TextInput(name="Tamanho", placeholder="P/M/G", sizing_mode="stretch_width"),
        "cor": pn.widgets.TextInput(name="Cor", placeholder="Cor", sizing_mode="stretch_width"),
    }

itens_widgets = create_itens_widgets()
btn_consultar_itens = pn.widgets.Button(name='Consultar', button_type='primary')
btn_inserir_itens = pn.widgets.Button(name='Inserir', button_type='success')
btn_atualizar_itens = pn.widgets.Button(name='Atualizar', button_type='warning')
btn_deletar_itens = pn.widgets.Button(name='Deletar', button_type='danger')

btn_consultar_itens.margin = (0, 1, 0, 10)
btn_inserir_itens.margin = (0, 1, 0, 0)
btn_atualizar_itens.margin = (0, 1, 0, 0)
btn_deletar_itens.margin = (0, 1, 0, 0)

def query_all_itens():
    try:
        query = """
        SELECT i.id_item, i.id_doacao, i.nome, i.descricao, i.estado_conservacao, 
               i.peso, i.volume, i.tamanho, i.cor,
               COALESCE(iod.id_ordem_doacao::text, 'Sem ordem') AS ordem_associada
        FROM item_doacao i
        LEFT JOIN item_ordem_doacao iod ON i.id_item = iod.id_item
        ORDER BY i.id_doacao DESC
        """
        df = pd.read_sql_query(query, engine)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro: {str(e)}', alert_type='danger')

def on_consultar_itens(w):
    try:
        id_item = w["id_item"].value
        if id_item == 0:
            query = """
            SELECT i.id_item, i.id_doacao, i.nome, i.descricao, i.estado_conservacao, 
                   i.peso, i.volume, i.tamanho, i.cor,
                   COALESCE(iod.id_ordem_doacao::text, 'Sem ordem') AS ordem_associada
            FROM item_doacao i
            LEFT JOIN item_ordem_doacao iod ON i.id_item = iod.id_item
            ORDER BY i.id_doacao DESC
            """
            df = pd.read_sql_query(query, engine)
        else:
            query = """
            SELECT i.id_item, i.id_doacao, i.nome, i.descricao, i.estado_conservacao, 
                   i.peso, i.volume, i.tamanho, i.cor,
                   COALESCE(iod.id_ordem_doacao::text, 'Sem ordem') AS ordem_associada
            FROM item_doacao i
            LEFT JOIN item_ordem_doacao iod ON i.id_item = iod.id_item
            WHERE i.id_item=%s
            """
            df = pd.read_sql_query(query, engine, params=(id_item,))
        
        if df.empty:
            return pn.pane.Alert('Nenhum item encontrado', alert_type='warning')
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro: {str(e)}', alert_type='danger')

def on_inserir_itens(w):
    try:
        if not w["id_doacao"].value or not w["nome"].value:
            pn.state.notifications.warning('Doação e Nome são obrigatórios!')
            return query_all_itens()
        
        cursor = con.cursor()
        cursor.execute(
            """INSERT INTO item_doacao(id_doacao, nome, descricao, estado_conservacao, peso, volume, tamanho, cor) 
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
            (w["id_doacao"].value, w["nome"].value, w["descricao"].value, w["estado_conservacao"].value,
             w["peso"].value, w["volume"].value, w["tamanho"].value, w["cor"].value)
        )
        con.commit()
        
        # Se uma ordem foi selecionada, criar a ligação
        if w["id_ordem_doacao"].value:
            cursor.execute(
                "SELECT id_item FROM item_doacao ORDER BY id_item DESC LIMIT 1"
            )
            id_item_novo = cursor.fetchone()[0]
            
            cursor.execute(
                """INSERT INTO item_ordem_doacao(id_item, id_ordem_doacao) 
                   VALUES (%s, %s)""",
                (id_item_novo, w["id_ordem_doacao"].value)
            )
            con.commit()
            pn.state.notifications.success('Item inserido e vinculado à ordem!')
        else:
            pn.state.notifications.success('Item inserido com sucesso!')
        
        cursor.close()
        return query_all_itens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'Erro ao inserir: {str(e)}', alert_type='danger')

def on_atualizar_itens(w):
    if not w["id_item"].value:
        pn.state.notifications.warning('ID do Item é obrigatório!')
        return query_all_itens()
    
    try:
        cursor = con.cursor()
        cursor.execute(
            """UPDATE item_doacao SET id_doacao=%s, nome=%s, descricao=%s, estado_conservacao=%s, 
                                    peso=%s, volume=%s, tamanho=%s, cor=%s WHERE id_item=%s""",
            (w["id_doacao"].value, w["nome"].value, w["descricao"].value, w["estado_conservacao"].value,
             w["peso"].value, w["volume"].value, w["tamanho"].value, w["cor"].value, w["id_item"].value)
        )
        con.commit()
        
        # Se uma ordem foi selecionada, atualizar a ligação
        if w["id_ordem_doacao"].value:
            # Remover ligação anterior
            cursor.execute(
                "DELETE FROM item_ordem_doacao WHERE id_item=%s",
                (w["id_item"].value,)
            )
            # Criar nova ligação
            cursor.execute(
                """INSERT INTO item_ordem_doacao(id_item, id_ordem_doacao) 
                   VALUES (%s, %s) 
                   ON CONFLICT (id_item, id_ordem_doacao) DO NOTHING""",
                (w["id_item"].value, w["id_ordem_doacao"].value)
            )
            con.commit()
            pn.state.notifications.success('Item atualizado e vinculado à ordem!')
        else:
            # Remover ligação se nenhuma ordem foi selecionada
            cursor.execute(
                "DELETE FROM item_ordem_doacao WHERE id_item=%s",
                (w["id_item"].value,)
            )
            con.commit()
            pn.state.notifications.success('Item atualizado!')
        
        cursor.close()
        return query_all_itens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'Erro ao atualizar: {str(e)}', alert_type='danger')

def on_deletar_itens(w):
    if not w["id_item"].value:
        pn.state.notifications.warning('ID do Item é obrigatório!')
        return query_all_itens()
    
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM item_doacao WHERE id_item=%s", (w["id_item"].value,))
        con.commit()
        cursor.close()
        pn.state.notifications.success('Item deletado com sucesso!')
        return query_all_itens()
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        return pn.pane.Alert(f'Erro ao deletar: {str(e)}', alert_type='danger')

def table_creator_itens(cons, ins, atu, dele, w):
    if cons: return on_consultar_itens(w)
    if ins: return on_inserir_itens(w)
    if atu: return on_atualizar_itens(w)
    if dele: return on_deletar_itens(w)
    return query_all_itens()

def create_interactive_table_itens():
    return pn.bind(table_creator_itens, btn_consultar_itens, btn_inserir_itens, btn_atualizar_itens, btn_deletar_itens, itens_widgets)

def create_itens_view(w):
    return pn.Row(
        pn.Column(
            pn.pane.Markdown("### 🎯 Gerenciar Itens de Doação"),
            create_btn_voltar(),
            w["id_item"],
            w["id_doacao"],
            w["id_ordem_doacao"],
            w["nome"],
            w["descricao"],
            w["estado_conservacao"],
            w["peso"],
            w["volume"],
            w["tamanho"],
            w["cor"],
            pn.Row(btn_consultar_itens, btn_inserir_itens, btn_atualizar_itens, btn_deletar_itens),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table_itens(), sizing_mode='stretch_both')
    )

def go_to_itens():
    main_area.clear()
    main_area.append(create_itens_view(itens_widgets))

In [ ]:
# ===== TELA DE DOAÇÕES =====

def get_doadores():
    """Busca todos os doadores já cadastrados"""
    try:
        query = """
        SELECT DISTINCT u.cpf_cnpj, u.nome
        FROM usuario u
        JOIN doador d ON u.cpf_cnpj = d.cpf_cnpj_d
        ORDER BY u.nome
        """
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['nome'], df['cpf_cnpj']))
    except Exception as e:
        return {}

def get_campanhas_ativas():
    """Busca apenas campanhas com status Ativa"""
    try:
        query = """
        SELECT
            c.id_campanha,
            CONCAT(u.nome, ' - ', c.nome) AS campanha_nome
        FROM campanha c
        INNER JOIN instituicao i
         ON c.cpf_cnpj_i = i.cpf_cnpj_i
        INNER JOIN usuario u
            ON u.cpf_cnpj = i.cpf_cnpj_i
        WHERE c.status = 'Ativa'
        ORDER BY u.nome, c.nome;
        """
        
        df = pd.read_sql_query(query, engine)
        return dict(zip(df['campanha_nome'], df['id_campanha']))
    except Exception as e:
        return {}

# Widgets para doações
doadores_dict = get_doadores()
doadores_select = pn.widgets.Select(
    name="Doador",
    options=doadores_dict,
    sizing_mode="stretch_width"
)

campanhas_ativas_dict = get_campanhas_ativas()
campanhas_select = pn.widgets.Select(
    name="Campanha Ativa",
    options=campanhas_ativas_dict,
    sizing_mode="stretch_width"
)

data_doacao = pn.widgets.DatePicker(
    name="Data da Doação",
    sizing_mode="stretch_width"
)

# Botões de ações para doações
btn_consultar_doacoes = pn.widgets.Button(name='Consultar', button_type='primary')
btn_registrar_doacoes = pn.widgets.Button(name='Inserir', button_type='success')
btn_atualizar_doacoes = pn.widgets.Button(name='Atualizar', button_type='warning')
btn_excluir_doacoes = pn.widgets.Button(name='Excluir', button_type='danger')

def on_consultar_doacoes(event):
    """Consulta todas as doações registradas"""
    try:
        query = """
        SELECT d.id_doacao, u.nome AS doador, c.nome AS campanha, d.data_doacao
        FROM doacao d
        JOIN doador doe ON d.cpf_cnpj_d = doe.cpf_cnpj_d
        JOIN usuario u ON doe.cpf_cnpj_d = u.cpf_cnpj
        JOIN campanha c ON d.id_campanha = c.id_campanha
        ORDER BY d.data_doacao DESC
        """
        df = pd.read_sql_query(query, engine)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro ao consultar: {str(e)}', alert_type='danger')

def on_registrar_doacao(event):
    """Registra uma nova doação"""
    try:
        if not doadores_select.value or not campanhas_select.value:
            pn.state.notifications.warning('Doador e Campanha Ativa são obrigatórios!')
            return
        
        if not data_doacao.value:
            pn.state.notifications.warning('Data da doação é obrigatória!')
            return
        
        cursor = con.cursor()
        
        # Verifica se a campanha está realmente ativa
        cursor.execute(
            "SELECT status FROM campanha WHERE id_campanha=%s AND status='Ativa'",
            (campanhas_select.value,)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Esta campanha não está mais ativa!')
            return
        
        # Registra a doação
        cursor.execute(
            """INSERT INTO doacao(data_doacao, cpf_cnpj_d, id_campanha) 
               VALUES (%s, %s, %s)""",
            (data_doacao.value, doadores_select.value, campanhas_select.value)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Doação registrada com sucesso!')
        
        # Limpar campos
        data_doacao.value = None
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao registrar: {str(e)}')

def on_atualizar_doacao(event):
    """Atualiza uma doação existente"""
    try:
        if not doadores_select.value or not campanhas_select.value:
            pn.state.notifications.warning('Doador e Campanha Ativa são obrigatórios!')
            return
        
        if not data_doacao.value:
            pn.state.notifications.warning('Data da doação é obrigatória!')
            return
        
        cursor = con.cursor()
        
        # Busca doação recente do doador para essa campanha
        cursor.execute(
            """SELECT id_doacao FROM doacao 
               WHERE cpf_cnpj_d=%s AND id_campanha=%s
               ORDER BY data_doacao DESC LIMIT 1""",
            (doadores_select.value, campanhas_select.value)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Nenhuma doação encontrada para este doador nesta campanha!')
            return
        
        # Atualiza a doação
        cursor.execute(
            """UPDATE doacao SET data_doacao=%s 
               WHERE id_doacao=%s""",
            (data_doacao.value, result[0])
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Doação atualizada com sucesso!')
        
        # Limpar campos
        data_doacao.value = None
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao atualizar: {str(e)}')

def on_excluir_doacao(event):
    """Exclui uma doação"""
    try:
        if not doadores_select.value or not campanhas_select.value:
            pn.state.notifications.warning('Doador e Campanha Ativa são obrigatórios!')
            return
        
        cursor = con.cursor()
        
        # Busca doação recente do doador para essa campanha
        cursor.execute(
            """SELECT id_doacao FROM doacao 
               WHERE cpf_cnpj_d=%s AND id_campanha=%s
               ORDER BY data_doacao DESC LIMIT 1""",
            (doadores_select.value, campanhas_select.value)
        )
        result = cursor.fetchone()
        
        if not result:
            cursor.close()
            pn.state.notifications.warning('Nenhuma doação encontrada para excluir!')
            return
        
        # Deleta a doação
        cursor.execute(
            "DELETE FROM doacao WHERE id_doacao=%s",
            (result[0],)
        )
        con.commit()
        cursor.close()
        pn.state.notifications.success('Doação excluída com sucesso!')
        
        # Limpar campos
        data_doacao.value = None
        
    except Exception as e:
        con.rollback()
        if 'cursor' in locals():
            cursor.close()
        pn.state.notifications.error(f'Erro ao excluir: {str(e)}')

btn_consultar_doacoes.on_click(on_consultar_doacoes)
btn_registrar_doacoes.on_click(on_registrar_doacao)
btn_atualizar_doacoes.on_click(on_atualizar_doacao)
btn_excluir_doacoes.on_click(on_excluir_doacao)

def create_doacoes_view():
    """Cria a tela de doações"""
    doacoes_layout = pn.Column(
        pn.pane.Markdown("### 🎁 Registrar Doação"),
        create_btn_voltar(),
        pn.pane.Markdown("**Selecione um doador e uma campanha ativa para registrar uma doação**"),
        doadores_select,
        campanhas_select,
        data_doacao,
        pn.Row(btn_consultar_doacoes, btn_registrar_doacoes, btn_atualizar_doacoes, btn_excluir_doacoes),
        pn.Spacer(height=20),
        pn.bind(on_consultar_doacoes, btn_consultar_doacoes),
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20, 20, 20, 20)
    )
    return doacoes_layout

def go_to_doacoes():
    main_area.clear()
    main_area.append(create_doacoes_view())

btn_consultar_doacoes.margin = (0, 3, 0, 10)
btn_registrar_doacoes.margin = (0, 3, 0, 0)
btn_atualizar_doacoes.margin = (0, 3, 0, 0)
btn_excluir_doacoes.margin = (0, 3, 0, 0)
